# OpenAI APIs Assignment: Prompt Engineering & Agent Pattern: Tool Use

## Assignment Overview
This assignment focuses on the following scenarios:
* Tool Use with Wikipedia: Implementing function calling with Wikipedia integration

## Assignment Structure: Tool Use with Wikipedia Integration 
Objective: Create a research assistant that can search Wikipedia and provide comprehensive information using OpenAI's function calling capabilities.

## Requirements:
* Wikipedia Search Function: Implement a function to search Wikipedia articles
* Content Extraction: Extract and summarize article content
* Function Calling: Use OpenAI's function calling to integrate Wikipedia data
* Structured Output: Return information in a structured format
* Error Handling: Handle cases where articles don't exist or are ambiguous

## Key Implementation Tasks:

```# Required libraries
import wikipedia
import openai
from dotenv import load_dotenv

# Function 1: Wikipedia Search
def search_wikipedia(query):
    """Search Wikipedia for articles related to the query"""
    # Students implement Wikipedia search using wikipediaapi
    # Return structured results with titles and descriptions

# Function 2: Content Extraction  
def get_wikipedia_content(title):
    """Get full content of a Wikipedia article"""
    # Students implement content extraction
    # Handle missing articles and disambiguation

# Function 3: OpenAI Function Calling
def handle_wikipedia_research(query):
    """Complete function calling workflow"""
    # Students implement the full workflow:
    # 1. Send query to OpenAI with function definitions
    # 2. Handle function calls (search_wikipedia, get_wikipedia_content)
    # 3. Execute requested functions
    # 4. Send results back to OpenAI for final response
```

## You can use one of these test cases to show:
* "What is artificial intelligence?"
* "Tell me about the history of Singapore"
* "Explain quantum computing"


## How to submit your assignment to Canvas
Submit the code with the input and output as a pdf file into Canvas 


In [ ]:
#Here is an example of the Wikipedia Search Function and you may need to install the wikipedia library API
import wikipedia

def get_article(search_term):
    results = wikipedia.search(search_term)
    first_result = results[0]
    page = wikipedia.page(first_result, auto_suggest=False)
    return page.content

article = get_article("What is artificial intelligence?")
#print(article[:1000]) 
#article is very long, so let's just print a preview

from IPython.display import display, HTML

article = get_article("What is artificial intelligence?")
html_content = f"""
<div style="background-color: #f8f9fa; padding: 20px; border-radius: 8px; border-left: 4px solid #007bff;">
    <h3 style="color: #007bff; margin-top: 0;">What is artificial intelligence? Answer Preview</h3>
    <p style="line-height: 1.6; text-align: justify;">{article[:1000]}...</p>
    <small style="color: #6c757d;">(Showing first 1000 characters)</small>
</div>
"""
display(HTML(html_content))

In [ ]:
#Assignment 1 - Your Answer 
import os
import wikipedia
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, HTML

# Load environment variables
load_dotenv()
# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"), organization=os.getenv("OPENAI_API_ORG"))

# Configure Wikipedia
wikipedia.set_lang("en")

# ----------------------------
# Function 1: Wikipedia Search
# ----------------------------
def search_wikipedia(query):
    """Search Wikipedia for articles related to the query"""
    try:
        results = wikipedia.search(query, results=5)
        if not results:
            return {"error": "No articles found"}
        return {"results": [{"title": title} for title in results]}
    except Exception as e:
        return {"error": str(e)}

# ----------------------------
# Function 2: Content Extraction
# ----------------------------
def get_wikipedia_content(title):
    """Get full content of a Wikipedia article"""
    try:
        page = wikipedia.page(title, auto_suggest=False)
        summary = wikipedia.summary(title, sentences=5)
        return {
            "title": page.title,
            "summary": summary,
            "content": page.content[:1200] + "...",  # preview
            "url": page.url,
        }
    except wikipedia.DisambiguationError as e:
        return {"error": f"Disambiguation error. Options: {e.options[:5]}"}
    except wikipedia.PageError:
        return {"error": f"Page '{title}' not found"}
    except Exception as e:
        return {"error": str(e)}

# ----------------------------
# Function 3: OpenAI Orchestrator
# ----------------------------
def handle_wikipedia_research(query):
    """Complete function calling workflow"""

    functions = [
        {
            "name": "search_wikipedia",
            "description": "Search Wikipedia for relevant articles",
            "parameters": {
                "type": "object",
                "properties": {"query": {"type": "string"}},
                "required": ["query"],
            },
        },
        {
            "name": "get_wikipedia_content",
            "description": "Get detailed content from a Wikipedia article",
            "parameters": {
                "type": "object",
                "properties": {"title": {"type": "string"}},
                "required": ["title"],
            },
        },
    ]

    # Step 1: Ask OpenAI how to handle the query
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": query}],
        functions=functions,
        function_call="auto",
    )

    message = response.choices[0].message

    # Step 2: Execute function calls
    if message.function_call:
        fn_name = message.function_call.name
        args = eval(message.function_call.arguments)

        if fn_name == "search_wikipedia":
            results = search_wikipedia(args["query"])
            if "results" in results and results["results"]:
                first_title = results["results"][0]["title"]
                return get_wikipedia_content(first_title)
            return results

        elif fn_name == "get_wikipedia_content":
            return get_wikipedia_content(args["title"])

    return {"response": message.content}

# ----------------------------
# Example Usage
# ----------------------------
query = "What is artificial intelligence?"
result = handle_wikipedia_research(query)

# Display nicely in Jupyter/Notebook
if "content" in result:
    html_content = f"""
    <div style="background-color: #f8f9fa; padding: 20px; border-radius: 8px; border-left: 4px solid #007bff;">
        <h3 style="color: #007bff; margin-top: 0;">{result['title']}</h3>
        <p style="line-height: 1.6; text-align: justify;">{result['content']}</p>
        <a href="{result['url']}" target="_blank">Read full article</a>
        <br><small style="color: #6c757d;">(Showing preview only; first 1000 characters)</small>
    </div>
    """
    display(HTML(html_content))
else:
    print(result)